# Irit's data
### 1. age & gender
### 2. swan:
##### - questions
##### - two factors
##### - imputation
##### - scoring 
##### - cutoffs
### 3. depp
##### questions
##### imputations
##### scoring
##### cutoff ?
#### 4. Suicide parameters
##### Suicide Ideation question
###### Suicide Ideation binar parameter
##### Suicide behavior question
###### Suicide behavior binar parameter


In [1]:
import pandas as pd
from Variables import swan_m, rename_align, info_columns, compute_swan_scores 
df = pd.read_excel(r"data/ADHD.xlsx")
rename_align_r = {j:i for i, j in rename_align.items()}
df = df.rename(rename_align_r, axis=1)

In [2]:
df, params = compute_swan_scores(df)

In [ ]:
df[CBCL_anx].sum()
df[CBCL_with].sum() 

cbcl_int_rawscore = CBCL_anx + CBCL_with

cbcl_int_tscore = ?

In [ ]:
#df_original = df_original[info_columns + swan_m].rename(rename_align, axis=1)

In [ ]:
df['suicidal_behavior'] = df['t1_p_rawscore_cbcl_18'] > 0

df['suicidal_ideation'] = df['t1_p_rawscore_cbcl_91'] > 0



In [ ]:
df.loc[df['ratio_of_missing_swan_values'] > 0.5, ['swan_score', 'swan_sum']] = np.nan
df.loc[df['ratio_of_missing_cbcl_values'] > 0.5, [cbcl_int_rawscore cbcl_int_tscore CBCL_anx + CBCL_with]] = np.nan